In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Date    : 2018-04-25 11:21:48
# @Author  : YIYING LIN (${email})
# @Link    : ${link}
# @Version : $Id$

import os
import pandas as pd
import numpy as np
import sklearn
import scipy as sp
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

filepath_pos = './train/pos/'
filelist_pos = os.listdir(filepath_pos)

filepath_neg = './train/neg/'
filelist_neg = os.listdir(filepath_neg)


def load_file(path):
    with open(path,'r') as file:
        lines = file.readlines()
        #words =[word.strip().split() for word in lines]
        lines = lines[0].strip().lower()
        file.close
        return lines


def add_data(filepath, filelist):
    alist = []
    for i in filelist:
        path = os.path.join('%s%s' % (filepath, i))
        line = load_file(path)
        alist.append(line)
    return alist


def clean_data(data, stopwords):
    data = [re.sub(r'\(|\)|\?|<br />|&|"|[0-9]|\n,', '', i) for i in data]
    data1 = []
    for sens in data:
        sen = ' '.join([i.replace('.', '')
                    for i in sens.split() if i.replace('.', '') not in stop])
        data1.append(sen)
    return data1

list_neg = add_data(filepath_neg, filelist_neg)
label_neg = [0] * len(list_neg)
list_pos = add_data(filepath_pos, filelist_pos)
label_pos = [1] * len(list_pos)

list_all = list_neg + list_pos
label_all = label_neg + label_pos

stop = stopwords.words('english') + list(string.punctuation)

list_all1 = clean_data(list_all,stop)
vectorizer = TfidfVectorizer(max_features=20000)
tfidf = vectorizer.fit_transform(list_all1).toarray()
# print(tfidf.shape)

columns = vectorizer.get_feature_names()
X = pd.DataFrame(tfidf, columns=columns)
Y = label_all

train_X, test_X, train_Y, test_Y = train_test_split(
    X, Y, test_size=0.2, random_state=1)
classifier = MultinomialNB()
fit = classifier.fit(train_X, train_Y)
pred = fit.predict(test_X)
accuracy = metrics.accuracy_score(test_Y, pred)
print (accuracy)


test = []
with open('test_data', 'r') as file:
    lines = file.readlines()
    for line in lines:
        test.append(line)
file.close

test_x = clean_data(test,stop)
tfidf_test = vectorizer.transform(test_x).toarray()
test_x = pd.DataFrame(tfidf_test,columns=columns)
pred_test = fit.predict(test_x)

with open('predict.txt','w') as output:
    for i in pred_test:
       output.write('%s\n'%i)
  
print(pred_test)
print(len(pred_test))

MemoryError: 

In [ ]:
上面程序输出的预测值 经过测试集评测（效果如下）：   
accuracy 0.83072   
precision 0.870430107527    
recall 0.77712   

In [7]:
print(vectorizer)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
